We are trying to evaluate how big of a deal pointing shifts are with respect to DEM results. We want to examine the actual change in NuSTAR DEM inputs related to changes in the region for a range of geometries related to pointing shifts – smearing, double centers – for a range of source brightnesses. Some of these are:

- 

In [1]:
import astropy.time
import numpy as np
from matplotlib import pyplot as plt

path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import nustar_dem_prep as nu
import region_fitting as rf


import importlib
import pathlib
from astropy.io import fits
import nustar_pysolar as nustar
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
from astropy import coordinates as coord


In [2]:
working_dir='./shift_region_tests/'

#Make a new working directory for prepped data/etc if it doesn't yet exist
save_path = pathlib.Path(working_dir)
if not save_path.exists():
    save_path.mkdir()

In [3]:
def percents(evt_file, regfile):
    with fits.open(evt_file) as hdu:
        evt_data = hdu[1].data
        hdr = hdu[1].header
    
    lowbounds=[0.,2.5,3.5,6.]#,10.]
    
    for low in lowbounds:
        cleanevt = nustar.filter.event_filter(evt_data, energy_low=low, energy_high=20.,
                                             no_bad_pix_filter=True, no_grade_filter=True)
        #print(len(cleanevt), len(evt_data))
        nustar_map = nustar.map.make_sunpy(cleanevt, hdr)
    
    
        offset, rad = rf.read_regfile(regfile, time_interval[0], time_interval[1], 'hourangle')
        region = CircleSkyRegion(
                        center = coord.SkyCoord(offset[0], offset[1], frame=nustar_map.coordinate_frame),
                        radius = rad
                    )
        
        regdata = rf.get_region_data(nustar_map, region, 0)
        percent = np.sum(regdata)/np.sum(nustar_map.data)
        print('Percent of >'+str(low)+' keV emission in region:', percent)

In [4]:
#Bright AR double center during shift:

time_interval = [astropy.time.Time('2021-04-29T18:27:55', scale='utc', format='isot'),
                    astropy.time.Time('2021-04-29T18:28:25', scale='utc', format='isot')]
time = time_interval
timestring = time[0].strftime('%H-%M-%S')
stopstring = time[1].strftime('%H-%M-%S')
timestring=timestring+'_'+stopstring
print(timestring)

datapath='/Users/jmdunca2/nustar/apr-2021/20615003001/'
obsid='20615003001'

gtifile=datapath+'event_cl/nu'+obsid+'A06_gti.fits'

regfile=path_to_dodem+'initial_dem_apr21/snowflakes/'+timestring+'/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg'
evt_file = path_to_dodem+'initial_dem_apr21/snowflakes/'+timestring+'/nu20615003001A06_0_4_p_cl_sunpos.evt'

percents(evt_file, regfile)

eng_tr=[[2.5,3.5], [3.5,6.], [6.,10.]]

res = nu.load_nustar(time_interval, eng_tr, working_dir, 'A', make_nustar=True, gtifile=gtifile, datapath=datapath, regfile=regfile, 
                edit_regfile=False, compare_fpm=False, combine_fpm=True, actual_total_counts=False, nofit=False,
                   use_fit_regfile=False, clobber=False, default_err=0.2, pile_up_corr=True, special_pha='', shush=True,
                   adjacent_grades=True, path_to_dodem=path_to_dodem)


dn_in=[]
edn_in=[]
chanax=[]
combine_fpm=True
rate, erate, nutrs, nu_tresp, nu_logt, fpm = res
for i in range(0, len(nutrs)):
    dn_in.append(rate[i])
    edn_in.append(erate[i])
    if combine_fpm==False:
        chanax.append(nutrs[i]+'-'+fpm)
    else:
        chanax.append(nutrs[i])
        
print('')
for i in np.arange(len(dn_in)):
    print(chanax[i],':    ',"{0:.2f}".format(dn_in[i]),
          "  {0:.2f}".format(edn_in[i]),
          " {0:.0f}".format(100.*edn_in[i]/dn_in[i]),'%')

18-27-55_18-28-25
Percent of >0.0 keV emission in region: 0.84603629417383
Percent of >2.5 keV emission in region: 0.8767462422634836
Percent of >3.5 keV emission in region: 0.8983364140480592
Percent of >6.0 keV emission in region: 0.9285714285714286
Using input regfile:
/Users/jmdunca2/do-dem/initial_dem_apr21/snowflakes/18-27-55_18-28-25/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg
Using grades 0-4 NuSTAR events.

For NuSTAR area, using region in:  /Users/jmdunca2/do-dem/initial_dem_apr21/snowflakes/18-27-55_18-28-25/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg
Max NuSTAR Energy:  10.320000648498535
Highest energy range: [6.0, 10.0]
Total Above:  1
Above Energies:  [10.32000065]

2.5-3.5keV :     17406.95   3494.78  20 %
3.5-6.0keV :     5819.32   1177.21  20 %
6.0-10.0keV :     238.89   59.71  25 %


In [48]:
#Bright AR double center during shift:

time_interval = [astropy.time.Time('2021-04-29T18:26:05', scale='utc', format='isot'),
                    astropy.time.Time('2021-04-29T18:26:35', scale='utc', format='isot')]
time = time_interval
timestring = time[0].strftime('%H-%M-%S')
stopstring = time[1].strftime('%H-%M-%S')
timestring=timestring+'_'+stopstring
print(timestring)

datapath='/Users/jmdunca2/nustar/apr-2021/20615003001/'
obsid='20615003001'

gtifile=datapath+'event_cl/nu'+obsid+'A06_gti.fits'

regfile=path_to_dodem+'initial_dem_apr21/'+timestring+'/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg'
evt_file = path_to_dodem+'initial_dem_apr21/'+timestring+'/nu20615003001A06_0_4_p_cl_sunpos.evt'



percents(evt_file, regfile)

eng_tr=[[2.5,3.5], [3.5,6.], [6.,10.]]

res = nu.load_nustar(time_interval, eng_tr, working_dir, 'A', make_nustar=True, gtifile=gtifile, datapath=datapath, regfile=regfile, 
                edit_regfile=False, compare_fpm=False, combine_fpm=True, actual_total_counts=False, nofit=False,
                   use_fit_regfile=False, clobber=False, default_err=0.2, pile_up_corr=True, special_pha='', shush=True,
                   adjacent_grades=True, path_to_dodem=path_to_dodem)


dn_in=[]
edn_in=[]
chanax=[]
combine_fpm=True
rate, erate, nutrs, nu_tresp, nu_logt, fpm = res
for i in range(0, len(nutrs)):
    dn_in.append(rate[i])
    edn_in.append(erate[i])
    if combine_fpm==False:
        chanax.append(nutrs[i]+'-'+fpm)
    else:
        chanax.append(nutrs[i])
        
print('')
for i in np.arange(len(dn_in)):
    print(chanax[i],':    ',"{0:.2f}".format(dn_in[i]),
          "  {0:.2f}".format(edn_in[i]),
          " {0:.0f}".format(100.*edn_in[i]/dn_in[i]),'%')

18-26-05_18-26-35
Percent of >0.0 keV emission in region: 0.8899196633511859
Percent of >2.5 keV emission in region: 0.9167887966129294
Percent of >3.5 keV emission in region: 0.95
Percent of >6.0 keV emission in region: 0.9690721649484536
Using input regfile:
/Users/jmdunca2/do-dem/initial_dem_apr21/18-26-05_18-26-35/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg
Using grades 0-4 NuSTAR events.
Now we will make some spectral data products.
ARF File:  []
RMF File:  []
PHA File:  []
ARF File:  []
RMF File:  []
PHA File:  []
ARF File:  []
RMF File:  []
PHA File:  []
100% [........................................................] 488180 / 488180

ARF File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_0_4_p_sr.arf']
RMF File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_0_4_p_sr.rmf']
PHA File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_0_4_p_sr.pha']
ARF File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_21_24_p_sr.arf']
RMF File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_21_24_p_sr.rmf']
PHA File:  ['./shift_region_tests/18-26-05_18-26-35/nu20615003001A06_21_24_p_sr.pha']

For NuSTAR area, using region in:  /Users/jmdunca2/do-dem/initial_dem_apr21/18-26-05_18-26-35/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg
Max NuSTAR Energy:  9.079999923706055

2.5-3.5keV :     24904.08   4998.20  20 %
3.5-6.0keV :     9472.26   1911.79  20 %
6.0-10.0keV :     426.68   101.26  24 %


In [21]:
importlib.reload(nu)

clobber=True

gtifile=datapath+'event_cl/nu'+obsid+'A06_gti.fits'
nu.make_nustar_products(time_interval, 'A', gtifile, datapath, regfile, working_dir, edit_regfile=True,
                            pile_up_corr=True, clobber=clobber, path_to_dodem=path_to_dodem, dip_before_products=False)


ARF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_p_sr.arf']
RMF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_p_sr.rmf']
PHA File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_p_sr.pha']
ARF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_4_p_sr.arf']
RMF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_4_p_sr.rmf']
PHA File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_0_4_p_sr.pha']
ARF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_21_24_p_sr.arf']
RMF File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_21_24_p_sr.rmf']
PHA File:  ['./shift_region_tests/18-27-55_18-28-25/nu20615003001A06_21_24_p_sr.pha']
100% [........................................................] 488180 / 488180

'/Users/jmdunca2/do-dem/initial_dem_apr21/snowflakes/18-27-55_18-28-25/nu20615003001A06_0_4_p_cl_sunpos_COM_region.reg'